In [ ]:
%reload_ext autoreload
%autoreload 2

In [ ]:
import os
import sys
import warnings
warnings.filterwarnings('ignore')

curruser = os.environ.get('USER')

# sys.path.insert(0, '/opt/workspace/{user}/system/support_library/'.format(user=curruser))
# sys.path.insert(0, '/opt/workspace/{user}/libs/'.format(user=curruser))
# sys.path.insert(0, '/opt/workspace/{user}/system/labdata/lib/'.format(user=curruser))

# sys.path.insert(0, '/opt/workspace/{}/notebooks/Clickstream_Analytics/scr'.format(curruser))
sys.path.insert(0, '/opt/workspace/{user}/notebooks/support_library/'.format(user=curruser)) 
sys.path.insert(0, '/opt/workspace/{user}/libs/python3.5/site-packages/'.format(user=curruser))
sys.path.insert(0, '/opt/workspace/{user}/notebooks/labdata/lib/'.format(user=curruser))

# sys.path.insert(0, '/home/{}/notebooks/support_library/'.format(curruser)) 
# sys.path.insert(0, '/home/{}/python35-libs/lib/python3.5/site-packages/'.format(curruser))
# sys.path.insert(0, '/home/{}/notebooks/labdata/lib/'.format(curruser))

# from tqdm import tqdm
# from tqdm._tqdm_notebook import tqdm_notebook
# tqdm_notebook.pandas()

from pathlib import Path
import pandas as pd
import numpy as np
import re 
import joblib
import json
import subprocess
import tqdm
# pd.set_option('display.float_format', lambda x: '%.3f' % x)
# pd.set_option('display.max_colwidth', -1)
# pd.set_option('display.max_rows', None)
# pd.set_option('display.max_columns', None)

from spark_connector import SparkConnector
from sparkdb_loader import spark
# from spark_helper import SparkHelper
from connector import OracleDB
import pyspark
from pyspark import SparkContext, SparkConf, HiveContext
from pyspark.sql.functions import udf 
from pyspark.sql import functions as f
from pyspark.sql.window import Window
from pyspark.sql.types import *
from pyspark.sql.dataframe import DataFrame
from pyspark.ml import Pipeline, PipelineModel
from pyspark.ml.feature import StringIndexer

import loader as load

# from corpora_process import utils
import time
from datetime import datetime
from dateutil.relativedelta import relativedelta

In [ ]:
def show(self, n=10):
    return self.limit(n).toPandas()

pyspark.sql.dataframe.DataFrame.show = show

In [ ]:
LOCAL_PATH = './csv'
HDFS_PATH = '/user/{}'.format(curruser)

In [ ]:
def load_as_csv(dst_df, file_name, save_local=True):
    local_path = os.path.join(LOCAL_PATH, file_name)
    hdfs_path = os.path.join(HDFS_PATH, file_name)

    dst_df \
        .coalesce(1).write \
        .format('com.databricks.spark.csv') \
        .mode('overwrite') \
        .option('sep', ',') \
        .option('header', 'true') \
        .save(hdfs_path)

    if save_local:
        proc = subprocess.Popen(['hdfs', 'dfs', '-getmerge', hdfs_path, local_path])
        proc.communicate()
        print('Import to file:', local_path)
    else:
        print('Import to HDFS:', hdfs_path)

In [ ]:
conn_schema = 'sbx_team_digitcamp'

In [ ]:
%%time
sp = spark(schema='sklod_external_google_analytics',
           queue = 'ektov1-av_ca-sbrf-ru',
           process_label="anthony_",
           numofinstances=20, 
           numofcores=8,
           dynamic_alloc=False,
           kerberos_auth=False
          ) 
print(sp.sc.version)
hive = sp.sql

In [ ]:
schema = 'sbx_t_team_speech_analytics' #'dl_t_team_speech_analytics'
conn_schema = 'sbx_team_digitcamp'

# Telemarketing

In [ ]:
tbl = 'telemarketing'

In [ ]:
tele = hive.sql("select * from {}.{}".format(schema,tbl))

In [ ]:
tele.count()

In [ ]:
tele.filter("ucid is not Null").count()

In [ ]:
tele = tele.withColumn('phone_number', f.substring('dialednumber', -10, 10))
tele = tele.withColumn('call_dt', f.from_unixtime(f.round(f.unix_timestamp('interactiongmtstarttime')/60)*60, 'yyyy-MM-dd HH:mm:ss').cast(TimestampType()))
tele.printSchema()
tele.show()

In [ ]:
# camps inns + iskra phones
sql  = '''SELECT DISTINCT 
                  c.inn as inn
                 ,cast(c.active_flg as FLOAT) as active_flg
                 ,c.rko_open
                 --,to_date(cast(c.rko_open/1000 as TIMESTAMP)) as rko_open
                 ,c.registration_date as reg_dt
                 --,to_date(cast(c.registration_date/1000 as TIMESTAMP)) as reg_dt
                 ,c.tm_camp_last as tm_name
                 --,c.tm_date_last as tm_dt
                 ,cast(c.tm_date_last/1000 as TIMESTAMP) as tm_dt
                 ,c.tm_result_last as tm_result
                 ,a.phone as phone
            FROM sbx_team_digitcamp.SAS_A1_21_TE_COL_N a,
                 sbx_team_digitcamp.sas_y1_tumanov_inn_rko_tm c
            WHERE a.inn = c.inn
                  AND a.phone is not null
      '''
camp = hive.sql(sql)
camp = camp.withColumn('active_flg', f.col('active_flg').cast(IntegerType()))
camp = camp.withColumn('phone', f.substring('phone', -10, 10))
camp.printSchema()
camp.show()

In [ ]:
# set 10 hours difference between timestamps
days = 1
join_condition = (camp.phone == tele.phone_number) & \
                 (f.abs(f.unix_timestamp(camp.tm_dt) - f.unix_timestamp(tele.call_dt)) <= days*24*60*60)

new_data = camp.join(tele, join_condition)
# new_data = new_data.withColumnRenamed('inn', 'inn2')

# targets = hive.sql('''SELECT inn, fd_month, tm_date, target
#                               FROM {}.{}'''.format(INN_TABLE))

# targets = targets.withColumn('tm_date', f.col('tm_date').cast(t.DateType()))
# targets = targets.withColumn('inn', f.col('inn').cast(t.LongType()))

# cond = (f.to_date(new_data.tm_dt) == f.to_date(targets.tm_date)) & \
#        (new_data.inn2 == targets.inn)

# new_data = new_data.join(targets, cond)

In [ ]:
new_data.cache()

In [ ]:
new_data.select("interactionid", "ucid", "phone_number", "inn" ).filter("ucid is not Null").show()

In [ ]:
new_data.select('inn', 'phone').distinct().count()

In [ ]:
new_data.filter("ucid is not Null").select('inn', 'phone').distinct().count()

In [ ]:
res = new_data.select("interactionid", "ucid", "phone_number", "inn" )
res.registerTempTable("tmp_inn_cell")
hive.sql("drop table if exists {}.{} purge".format("sbx_team_digitcamp", "telemarketing_cell2inn"))
hive.sql("create table {}.{} select * from tmp_inn_cell".format("sbx_team_digitcamp", "telemarketing_cell2inn"))

In [ ]:
rep = hive.sql("select * from {}.db_sme_report_report_kc_all".format("sbx_internal_kakb_sas"))

In [ ]:
rep.take(1)

In [ ]:
rep.select(f.max(f.to_timestamp("CALL_DATE"))).show()

In [ ]:
rep.select("CALL_RESULT", "CALL_RESULT_EXT", "COMM", "CRM_ID", "INN").show()

# AIST_INTERACTION_HISTORY

In [ ]:
aist = hive.sql("select * from {}.AIST_INTERACTION_HISTORY".format(conn_schema))

In [ ]:
aist.select(f.max("INTERACTION_START_DATE").alias('max_interac')).show()

In [ ]:
aist.filter("ACTION_ITEM_TASK_CRM_ID is not Null").count()

In [ ]:
aist.dtypes

In [ ]:
aist = aist.withColumn("COALESCE_INTERACTION_COMMENTARY", f.when((f.col("INTERACTION_COMMENTARY")=='null')&
                                                                 (f.col("ACTION_ITEM_RESULT_COMMENTARY")!='null'),
                                                                 f.col("ACTION_ITEM_RESULT_COMMENTARY")
                                                                ).otherwise(f.col("INTERACTION_COMMENTARY")))\
           .withColumn("COALESCE_INTERACTION_REASON", f.when((f.col("INTERACTION_REASON")=='null')&
                                                             (f.col("ACTION_ITEM_RESULT_REASON")!='null'),
                                                                f.col("ACTION_ITEM_RESULT_REASON")
                                                            ).otherwise(f.col("INTERACTION_REASON")))

In [ ]:
aist.filter("INTERACTION_REASON =='null' and ACTION_ITEM_RESULT_REASON != 'null'")\
    .select("INTERACTION_REASON",
            "ACTION_ITEM_RESULT_REASON").show(50)

In [ ]:
aist.filter("INTERACTION_COMMENTARY == 'null' and ACTION_ITEM_RESULT_COMMENTARY != 'null'")\
    .select("INTERACTION_COMMENTARY",
            "ACTION_ITEM_RESULT_COMMENTARY",
            "COALESCE_INTERACTION_COMMENTARY",
            "COALESCE_INTERACTION_REASON").show(50)

In [ ]:
comments = aist.select("INTERACTION_ID", "COALESCE_INTERACTION_COMMENTARY").distinct().collect()

In [ ]:
reasons = aist.select("INTERACTION_ID", "COALESCE_INTERACTION_REASON").distinct().collect()

In [ ]:
len(comments), len(reasons)

## Export comments

In [ ]:
with open('./csv/COALESCE_INTERACTION_COMMENTARY', 'w+') as fout:
    for item in comments:
        if item['COALESCE_INTERACTION_COMMENTARY'] != None:
            item = item['COALESCE_INTERACTION_COMMENTARY']+'\n'
            fout.writelines(item)

In [ ]:
sdf = aist.filter("COALESCE_INTERACTION_COMMENTARY != 'null'").select("INTERACTION_ID", "COALESCE_INTERACTION_COMMENTARY")
load_as_csv(sdf,'km_comments.csv')

### Check DateTime for the First non Null CALL_UCID

In [ ]:
isucid_min_dttm = aist.filter('CALL_UCID is not Null')\
                      .select(f.min('INTERACTION_START_DATE').alias('isucid_min_dttm')).collect()[0]['isucid_min_dttm']

In [ ]:
isucid_min_dttm = isucid_min_dttm.split('.')[0]
# isucid_min_dttm = re.sub(r'\d{2}\:\d{2}\:\d{2}','00:00:00',isucid_min_dttm)
isucid_min_dttm

## Filter and Rebuild AIST Table

In [ ]:
dct = \
dict(list(zip(['ACTION_ITEM_ORG_ID',
               'CORPORATE_INTERACTION_ORG_ID',
               'ACTION_ITEM_TASK_CRM_ID',
               'ACTION_ITEM_PROD_OFFER_CRM_ID'], ['']*4)))

In [ ]:
aist = aist.fillna(dct)
aist = aist.replace(to_replace = {'null': ''}, subset = list(dct.keys())+['CALL_PHONE_NUMBER'])

In [ ]:
refdatetime = datetime.fromtimestamp(datetime.timestamp(datetime.now())-40*24*60*60)
refdate = datetime.strftime(refdatetime, format='%Y-%m-%d')

In [ ]:
aist_filter = \
aist.select('INTERACTION_ID',
            'CALL_ID',
            'CALL_UCID',
            #'CALL_EMPLOYEE_ID',
            #'CALL_AGENT_LOGIN',
            #'INTERACTION_LOGIN',
            #'ACTION_ITEM_LOGIN',
            #f.from_unixtime(f.round(f.unix_timestamp('CALL_START_DATE')/60)*60, 'yyyy-MM-dd HH:mm:ss').cast(TimestampType()).alias('CALL_START_DATE'),
            #f.from_unixtime(f.round(f.unix_timestamp('CALL_END_DATE')/60)*60, 'yyyy-MM-dd HH:mm:ss').cast(TimestampType()).alias('CALL_END_DATE'),
            f.from_unixtime(f.round(f.unix_timestamp('INTERACTION_START_DATE')/60)*60, 'yyyy-MM-dd HH:mm:ss').cast(TimestampType()).alias('INTERACTION_START_DATE'),
            f.from_unixtime(f.round(f.unix_timestamp('INTERACTION_END_DATE')/60)*60, 'yyyy-MM-dd HH:mm:ss').cast(TimestampType()).alias('INTERACTION_END_DATE'),           
            f.from_unixtime(f.round(f.unix_timestamp('ACTION_ITEM_CREATE_DATE')/60)*60, 'yyyy-MM-dd HH:mm:ss').cast(TimestampType()).alias('ACTION_ITEM_CREATE_DATE'),
            #f.from_unixtime(f.round(f.unix_timestamp('ACTION_ITEM_FACT_START_DATE')/60)*60, 'yyyy-MM-dd HH:mm:ss').cast(TimestampType()).alias('ACTION_ITEM_FACT_START_DATE'),
            #f.from_unixtime(f.round(f.unix_timestamp('ACTION_ITEM_FACT_END_DATE')/60)*60, 'yyyy-MM-dd HH:mm:ss').cast(TimestampType()).alias('ACTION_ITEM_FACT_END_DATE'),            
            'CALL_INCOMING_CALL',
            f.substring('CALL_PHONE_NUMBER',-10,10).alias('CALL_PHONE_NUMBER'),
            f.split(f.col('ACTION_ITEM_ORG_ID'),'_').getItem(0).alias('ACTION_ITEM_ORG_ID'),
            #f.split(f.col('CORPORATE_INTERACTION_ORG_ID'),'_').getItem(0).alias('CORPORATE_INTERACTION_ORG_ID'),
            f.split(f.col('ACTION_ITEM_TASK_CRM_ID'),'_').getItem(0).alias('ACTION_ITEM_TASK_CRM_ID'),
            f.split(f.col('ACTION_ITEM_PROD_OFFER_CRM_ID'),'_').getItem(0).alias('ACTION_ITEM_PROD_OFFER_CRM_ID'),
            'ACTION_ITEM_RESULT_PRODUCT_NAME',
            #'ACTION_ITEM_ID_PROD_OFFER_LITE',
            'ACTION_ITEM_PRODUCT_ID',
            'ACTION_ITEM_PRODUCT_DESCRIPTION',
            'COALESCE_INTERACTION_COMMENTARY',
            'COALESCE_INTERACTION_REASON'
            ).filter((f.col('CALL_INCOMING_CALL')==0)&
                     (f.col('CALL_PHONE_NUMBER')!='')&
                     (f.col('INTERACTION_START_DATE').cast(TimestampType()) >= f.to_timestamp(f.lit(refdate))) #f.lit(isucid_min_dttm).cast(TimestampType())
                    )

In [ ]:
aist_filter.cache()

In [ ]:
aist_filter.count()

In [ ]:
aist_filter.filter("ACTION_ITEM_RESULT_PRODUCT_NAME is not Null").count()

In [ ]:
aist_filter.columns

In [ ]:
aist_filter.filter("ACTION_ITEM_TASK_CRM_ID != '' ").select( 'ACTION_ITEM_ORG_ID',
                                                             'CORPORATE_INTERACTION_ORG_ID',
                                                             'ACTION_ITEM_TASK_CRM_ID',
                                                             'ACTION_ITEM_PROD_OFFER_CRM_ID',
                                                             'ACTION_ITEM_RESULT_PRODUCT_NAME',
                                                             'ACTION_ITEM_ID_PROD_OFFER_LITE',
                                                             'ACTION_ITEM_PRODUCT_ID',
                                                             'ACTION_ITEM_PRODUCT_DESCRIPTION',
                                                             'COALESCE_INTERACTION_COMMENTARY').show()

In [ ]:
aist_filter.filter("ACTION_ITEM_PROD_OFFER_CRM_ID is not Null").select("ACTION_ITEM_PROD_OFFER_CRM_ID", 
                                                                       "ACTION_ITEM_ORG_ID",
                                                                       "ACTION_ITEM_TASK_CRM_ID"
                                                                      ).show()

## MA_TASK

In [ ]:
matsk = hive.sql("select * from {}.MA_TASK".format(conn_schema))

In [ ]:
matsk.show()

In [ ]:
matsk_filter = matsk.filter("CREATE_DT>=timestamp('{}')".format(refdate)).select('CREATE_DT', 
                                                                                 'END_FACT_DT',
                                                                                 'HOST_CUST_ORG_ID',
                                                                                 'INN')

In [ ]:
matsk_filter.cache()

In [ ]:
matsk_filter.count()

### Check Intersection between MA_TASK and AIST by ORD_ID

In [ ]:
matsk_filter.filter('HOST_CUST_ORG_ID = "1-TWAUV1T"').show()

In [ ]:
aist_filter.filter('ACTION_ITEM_ORG_ID = "1-TWAUV1T"').show()

## Join Filtered AIST with MA_TASK

In [ ]:
# # set 2 hours difference between timestamps
# days = 1
# join_condition = ( (f.abs(f.unix_timestamp(aist_filter.CALL_START_DATE) - 
#                           f.unix_timestamp(matsk_filter.CREATE_DT))) <= days*24*60*60)|\
#                  (aist_filter.CALL_START_DATE >= matsk_filter.CREATE_DT)&\
#                  (aist_filter.CALL_START_DATE <= matsk_filter.END_FACT_DT)

# aist_filter_task = aist_filter.join(matsk_filter, on=join_condition, how='left_outer')

In [ ]:
days = 2
join_condition =  (
                      (f.abs(
                               f.unix_timestamp(aist_filter.INTERACTION_START_DATE) - 
                               f.unix_timestamp(matsk_filter.CREATE_DT)
                            ) <= days*24*60*60
                      )|\
                      ((aist_filter.INTERACTION_START_DATE >= matsk_filter.CREATE_DT)&\
                       (aist_filter.INTERACTION_START_DATE <= matsk_filter.END_FACT_DT)
                      )
                  )&\
                  (aist_filter.ACTION_ITEM_ORG_ID == matsk_filter.HOST_CUST_ORG_ID)

aist_filter_task = aist_filter.join(matsk_filter, on=join_condition, how='left_outer')

## Export Joined Table

In [ ]:
import subprocess

In [ ]:
aist_filter_task.registerTempTable("tmp_aist_task")
hive.sql("drop table if exists {}.{} purge".format("sbx_team_digitcamp", "aist_hist_matask_join"))
subprocess.call(['hdfs', 'dfs', '-rm', '-r', '-f', '-skipTrash', "hdfs://clsklsbx/user/team/team_digitcamp/hive/{}".format("aist_hist_matask_join")])

hive.sql("create table {}.{} select * from tmp_aist_task".format("sbx_team_digitcamp", "aist_hist_matask_join"))

## Check aist_hist_matask_join

In [ ]:
aistmatask = hive.sql('''select 
                         ACTION_ITEM_ORG_ID, 
                         ACTION_ITEM_PRODUCT_ID,
                         ACTION_ITEM_RESULT_PRODUCT_NAME,
                         COALESCE_INTERACTION_COMMENTARY,
                         CALL_UCID,
                         CALL_PHONE_NUMBER,
                         INTERACTION_START_DATE,
                         INTERACTION_END_DATE,
                         INN
                         from sbx_team_digitcamp.aist_hist_matask_join''')

In [ ]:
aistmatask.filter("INN is not Null").count()

In [ ]:
aistmatask.filter("CALL_UCID = '501073771594718344'").show(20)

In [ ]:
aistmatask.select(f.min("CALL_START_DATE")).show()

In [ ]:
aistmatask.filter((f.col("CALL_START_DATE") > pd.to_datetime('2020-01-14')) &
                  (f.col("CALL_START_DATE") < pd.to_datetime('2020-01-15')) &
                  (f.col("CALL_PHONE_NUMBER") == '9246656035')).show()

## Apply User Defined RegExp on comments field

In [ ]:
aistmatask = \
aistmatask.withColumn("isNeedForCall",
                        f.regexp_extract(f.col('COALESCE_INTERACTION_COMMENTARY'),
                                         '''(.*подумает.*|.*перезвон.*ближе.*|.*не\s*удобно.*договорились.*|
                                             .*перезвон.*|.*клиент просил.*|.*просил.*(?:набрать)*|.*позвони.*|.*(?!<не) интерес.*)''',1)
                     )

In [ ]:
aistmatask.filter("isNeedForCall !='' and INN is not Null").show(70)

In [ ]:
df = aistmatask.filter("isNeedForCall !='' and INN is not Null").toPandas()

In [ ]:
df.drop(columns=["CALL_UCID", "CALL_PHONE_NUMBER"], inplace=True)

In [ ]:
df.head()

In [ ]:
df.to_excel("./csv/aist_ckr_needforcall.xlsx", index=False, encoding='utf-8')

## Analyse results from NICE datamart: cell_inn_interaction_end_date

In [ ]:
topics = pd.read_excel('./csv/Nice_Topics.xlsx',header=None)

In [ ]:
topics.reset_index(inplace=True, level=0)
topics.rename(columns={'index':'topic_num',0:'topics'}, inplace=True)
topics.head()

In [ ]:
topics_sdf = hive.createDataFrame(topics, schema=StructType([StructField('topic_num', IntegerType()), 
                                                      StructField('topics', StringType())
                                                     ])
                          )

In [ ]:
dialogs = hive.sql('''select inn inn_tm, 
                             substring(ucid,-18,18) ucid, \
                             phone_number, date, topic_num from sbx_team_digitcamp.cell_inn_interaction_end_date''')

In [ ]:
dialogs = dialogs.join(topics_sdf, on='topic_num', how='left_outer')

In [ ]:
dialogs.cache()

In [ ]:
dialogs.filter("ucid is not Null").show()

In [ ]:
dialogs.filter("inn_tm is not Null").count()

In [ ]:
# dialogs = dialogs.select(f.col("inn_tm").alias("inn_tm_"), 
#                          f.col("phone_number").alias("phone_number_"), 
#                          f.col("topics").alias("topics_"))

## Join with AISTMATASK

In [ ]:
days = 2
join_condition =  (
                     ((f.abs(
                               f.unix_timestamp(aistmatask.INTERACTION_START_DATE) - 
                               f.unix_timestamp(dialogs.date)
                            ) <= days*24*60*60
                      )|\
                      ((dialogs.date >= aistmatask.INTERACTION_START_DATE)&\
                       (dialogs.date <= aistmatask.INTERACTION_END_DATE)
                      )
                     )&(dialogs.phone_number == aistmatask.CALL_PHONE_NUMBER) 
                  )|\
                  (dialogs.ucid == aistmatask.CALL_UCID)
    
dialogsaist = dialogs.join(aistmatask, on=join_condition, how='left_outer')

In [ ]:
dialogsaist.cache()

In [ ]:
cols = \
[
 'INN' ,
 'inn_tm',
 'ucid',
 'phone_number',
 'date',
 'topics',
 'ACTION_ITEM_ORG_ID',
 'ACTION_ITEM_PRODUCT_ID',
 'ACTION_ITEM_RESULT_PRODUCT_NAME',
 'INTERACTION_START_DATE',
 'INTERACTION_END_DATE'
]

In [ ]:
dialogsaist = dialogsaist.select(*[cols])
dialogsaist = dialogsaist.select([f.col(col).alias(col.upper()) for col in dialogsaist.columns])

In [ ]:
dialogsaist.registerTempTable("tmp_aist_dialogs")
hive.sql("drop table if exists {}.{} purge".format("sbx_team_digitcamp", "telemarketing_aist_cell2inn"))
subprocess.call(['hdfs', 'dfs', '-rm', '-r', '-f', '-skipTrash', "hdfs://clsklsbx/user/team/team_digitcamp/hive/{}".format("telemarketing_aist_cell2inn")])

hive.sql("create table {}.{} select * from tmp_aist_dialogs".format("sbx_team_digitcamp", "telemarketing_aist_cell2inn"))

## telemarketing_aist_cell2inn

In [ ]:
dialogsaist = hive.sql("select * from {}.telemarketing_aist_cell2inn".format(conn_schema))

In [ ]:
dialogsaist = \
dialogsaist.select(*[f.col(col).alias(col.split("upper(")[-1][:-1].upper()) for col in dialogsaist.columns])

In [ ]:
dialogsaist.show()

In [ ]:
dialogsaist.filter("(INN is not Null) or (INN_TM is not Null)").count()

In [ ]:
df = dialogsaist.toPandas()

In [ ]:
df.head()

In [ ]:
df.to_excel("./csv/inn_telem_topics.xlsx", index=False, encoding='utf-8')

## CALLS_UUP_DETAIL

In [ ]:
callsup = hive.sql("select * from {}.CALLS_UUP_DETAIL".format(conn_schema))

In [ ]:
callsup.show()

# Select records from nice table

In [ ]:
nice = hive.sql("select * from {}.nice".format(schema))

In [ ]:
nice = nice.withColumn('phone_number', f.substring('phonenumbers', -10, 10))
nice = nice.withColumn('call_dt', f.from_unixtime(f.round(f.unix_timestamp('interactiongmtstarttime')/60)*60, 'yyyy-MM-dd HH:mm:ss').cast(TimestampType()))
nice.printSchema()

In [ ]:
nice = nice.replace({'NULL':''},['ucid'])

In [ ]:
nice.show()

In [ ]:
# nice_aist = nice.filter("ucid=''")
nice_aist = nice.filter("call_dt<timestamp('{}')".format(isucid_min_dttm))
nice_aist.count()

In [ ]:
nice_aist.select(f.max('call_dt')).show()

In [ ]:
# set 2 hours difference between timestamps
days = 1
join_condition = ( (f.abs(f.unix_timestamp(nice_aist.call_dt) - 
                          f.unix_timestamp(uup_pp_part.create_date))) <= days*24*60*60)

new_data = reswithaist_part.join(uup_pp_part, join_condition)

## Join by USID with AIST records

In [ ]:
aist_filter.count()

In [ ]:
join_condition = (enter.ucid == aist_filter.call_ucid)
reswithaist = enter.join(aist_filter, join_condition)

In [ ]:
reswithaist.count()

### Select data from additional iskra tables

In [ ]:
calls_upp = hive.sql("select * from {}.CALLS_UUP_DETAIL".format(schema))
uup_pp = hive.sql("select * from {}.FV_CKR_UUP_PP".format(schema))
uup_val = hive.sql("select * from {}.FV_CKR_UUP_VAL".format(schema))

In [ ]:
calls_upp = calls_upp.select([f.col(col).alias(col.lower()) for col in calls_upp.columns])
calls_upp = calls_upp.withColumn('tab', f.col('tab_').cast(IntegerType()))
calls_upp = calls_upp.drop('tab_')
calls_upp.printSchema()

In [ ]:
uup_pp = uup_pp.select([f.col(col).alias(col.lower()) for col in uup_pp.columns])
uup_pp = uup_pp.withColumn('tab', f.col('tab_num').cast(IntegerType()))
uup_pp = uup_pp.drop('tab_num')
uup_pp.printSchema()

In [ ]:
uup_pp.select(f.col('inn').cast(IntegerType()).cast(StringType()),
              'offer_id',
              'org_id',
              'product_id', 
              'product_group', 
              'product_name').show()

### AIST: Find relevant INN base don the join with FV_CKR_UUP_PP

In [ ]:
reswithaist.dtypes

In [ ]:
ref_date_down = '2019-08-01 00:00:00'
ref_date_up = '2020-03-17 00:00:00'
reswithaist_part = reswithaist.filter('''call_dt >= timestamp("{down}") and 
                                         call_dt <= timestamp("{up}")'''.format(down=ref_date_down,
                                                                                up=ref_date_up))
uup_pp_part = uup_pp.filter('''create_date >= timestamp("{down}") and 
                               create_date <= timestamp("{up}")'''.format(down=ref_date_down,
                                                                          up=ref_date_up))

In [ ]:
# set 2 hours difference between timestamps
days = 1
join_condition = (reswithaist_part.action_item_product_id == uup_pp_part.product_id)&\
                 ( (f.abs(f.unix_timestamp(reswithaist_part.call_dt) - 
                          f.unix_timestamp(uup_pp_part.create_date))) <= days*2*60*60)

new_data = reswithaist_part.join(uup_pp_part, join_condition)

In [ ]:
new_data.drop_duplicates()
new_data.count()

In [ ]:
new_data.show()

In [ ]:
new_data.filter('ucid = "00501038731580989146"').select('inn',
                                                        'product_id',
                                                        'action_item_product_id',
                                                        'product_name',
                                                        'action_item_result_product_name',
                                                        'call_dt',
                                                        'create_date'
                                                        ).show(30)

### Total match based on the CALLS_UUP_DETAIL datamart

In [ ]:
ref_date_down = '2019-08-01 00:00:00'
ref_date_up = '2020-03-17 00:00:00'
calls_upp = calls_upp.filter('''row_datetime >= timestamp("{down}") and 
                                row_datetime <= timestamp("{up}")'''.format(down=ref_date_down,
                                                                            up=ref_date_up))
uup_pp = uup_pp.filter('''create_date >= timestamp("{down}") and 
                          create_date <= timestamp("{up}")'''.format(down=ref_date_down,
                                                                            up=ref_date_up))

In [ ]:
calls_upp.count(), uup_pp.count()

In [ ]:
calls_upp.show()

In [ ]:
# set 2 hours difference between timestamps
days = 1
join_condition = (calls_upp.tab == uup_pp.tab)&\
                 ( (f.abs(f.unix_timestamp(calls_upp.row_datetime) - 
                          f.unix_timestamp(uup_pp.create_date))) <= days*2*60*60)

new_data = calls_upp.join(uup_pp, join_condition)

In [ ]:
new_data = new_data.drop_duplicates()
new_data.cache()

In [ ]:
new_data.count()
#105208897

### Join with nice

In [ ]:
# days = 3
# join_condition = (f.round(f.abs(f.unix_timestamp(enter.call_dt) - f.unix_timestamp(new_data.CREATE_DATE))/60) <= days*24*60)
join_condition = (enter.ucid == new_data.ucid)
res = enter.join(new_data, join_condition)

In [ ]:
res = res.drop_duplicates()
res.count()

In [ ]:
res.show()